In [1]:
import pandas as pd
import anal_preprocess as ap
import seaborn as sns
import matplotlib.pyplot as plt
import get_numcatg_data as gnca
pd.set_option('display.max_columns', None)
df = pd.read_csv('./student-mat.csv')
df_ori = df.copy()
df['grade'] = df['G1'].values + df['G2'].values + df['G3'].values 
input_data = df.drop(columns=['grade'])

#### 데이터 전처리

In [2]:
mdsource = gnca.GetNumericCategoryAllData(df, df)
col_out = "grade"
numeric_data, catg_data, ctg_data = mdsource.main(col_out)

ShapiroResult(statistic=0.9926792337421895, pvalue=0.05052375511183993)
본 결측치 체크는 연속형 변수에 대하여 진행합니다.
결측치 데이터 컬럼 없음!
Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3', 'grade'],
      dtype='object')
이상치를 결측치로 대체한 후 KNN 방식으로 대체 ... 2차결측치 검증 진행
본 결측치 체크는 연속형 변수에 대하여 진행합니다.
결측치 데이터 컬럼 : 6개, ['age', 'traveltime', 'studytime', 'failures', 'Dalc', 'absences'] 
age            1
Medu           0
Fedu           0
traveltime     8
studytime     27
failures      83
famrel         0
freetime       0
goout          0
Dalc          18
Walc           0
health         0
absences      15
G1             0
G2             0
G3             0
grade          0
dtype: int64
KNN 알고리즘 대체
결측치 대체 후 검정 결측치 컬럼갯수 : 0 
본 결측치 체크는 연속형 변수에 대하여 진행합니다.
결측치 데이터 컬럼 : 4개, ['Fjob_encd', 'guardian_encd', 'reason_encd', 'Mjob_encd'] 
Fjob_encd        3
guardian_encd    2
reason_encd      3
Mjob_encd        2
dt

In [3]:
len(catg_data.columns)

17

In [4]:
diff_catg_cols = set(ctg_data.columns).difference(set(catg_data.columns))
category_data = ctg_data[list(diff_catg_cols)]

In [5]:
df = pd.concat([numeric_data, category_data], axis=1)
df.isna().sum()

age                0
Medu               0
Fedu               0
traveltime         0
studytime          0
failures           0
famrel             0
freetime           0
goout              0
Dalc               0
Walc               0
health             0
absences           0
G1                 0
G2                 0
G3                 0
grade              0
romantic_encd      0
famsize_encd       0
higher_encd        0
address_encd       0
sex_encd           0
guardian_encd      0
famsup_encd        0
schoolsup_encd     0
paid_encd          0
activities_encd    0
nursery_encd       0
Mjob_encd          0
internet_encd      0
school_encd        0
Pstatus_encd       0
Fjob_encd          0
reason_encd        0
dtype: int64

In [6]:
#### 데이터 분할
from sklearn.model_selection import train_test_split
x = df.drop(col_out, axis=1)
y = df[col_out]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=2022)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(276, 33)
(119, 33)
(276,)
(119,)


### svm, xgboost, randomforest 3개 알고리즘 
#### 회귀분석과 분류분석에 모두 적용가능하다 / 범주형 변수를 독립변수로 사용할 수 없다 / 과대과소적합을 피할 수 있음 / 회귀분석의 다중공선성의 문제를 해결

In [8]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [9]:
scaler = StandardScaler()
x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_test_scaled = pd.DataFrame(scaler.fit_transform(x_test), columns=x_test.columns)

#### SVM

In [10]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'C':[0.1, 1, 10, 100], 'gamma':[0.001, 0.01, 0.1, 1, 10]}
]
grid_svm = GridSearchCV(SVR(), param_grid = param_grid, cv =5)
grid_svm.fit(x_train_scaled, y_train)
result=pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_socre'] = grid_svm.cv_results_['mean_test_score']
result = result.sort_values(by='mean_test_socre', ascending=False)
result = result.reset_index()
c_value = result.loc[0,"C"]
g_value = result.loc[0,"gamma"]
import numpy as np
svr = SVR(C=c_value, gamma=g_value)
svr.fit(x_train_scaled, y_train)
print('R2: ', svr.score(x_test_scaled, y_test))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, svr.predict(x_test_scaled))))

R2:  0.9968953818705566
RMSE:  0.6236536786319598


### RandomForest

In [11]:
rf_grid = [
    {'max_depth':[2,4,6,8,10], 'min_samples_split':[2,4,6,7,10]}
]
rf = GridSearchCV(RandomForestRegressor(n_estimators=100), param_grid=rf_grid, cv=5)
rf.fit(x_train, y_train)
print('R2: ', rf.score(x_test, y_test))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, rf.predict(x_test_scaled))))

R2:  0.9924719585498518
RMSE:  28.509888877242524


### XGBoost

In [12]:
xgb_grid = [
    {'max_depth':[2,4,6,8,10], 'min_samples_split':[2,4,6,7,10]}
]
xgb = GridSearchCV(XGBRegressor(n_estimators=100), param_grid=xgb_grid, cv=5)
xgb.fit(x_train, y_train)
print('R2: ', xgb.score(x_test, y_test))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, xgb.predict(x_test_scaled))))

R2:  0.9958258959313705
RMSE:  28.195070842525038
